Q1. Downloading the data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2022.

Read the data for January. How many columns are there?

A: 19

In [1]:
!wget "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet"
!wget "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet"

--2023-05-16 15:38:42--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.64.100.55, 18.64.100.144, 18.64.100.74, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.64.100.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38139949 (36M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2022-01.parquet’

yellow_tripdata_202 100%[===================>]  36,37M  5,18MB/s    in 8,4s    

2023-05-16 15:38:50 (4,34 MB/s) - ‘yellow_tripdata_2022-01.parquet’ saved [38139949/38139949]

--2023-05-16 15:38:51--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.64.100.55, 18.64.100.144, 18.64.100.74, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.64.100.55|:443... failed: Conne

In [2]:
!mv yellow_tripdata_2022-01.parquet data/
!mv yellow_tripdata_2022-02.parquet data/

In [3]:
import pandas as pd
import numpy as np
import pickle

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso

from sklearn.metrics import mean_squared_error

In [71]:
df = pd.read_parquet('./data/yellow_tripdata_2022-01.parquet')
# Using .shape
num_of_columns = df.shape[1]
print('Number of columns using shape:', num_of_columns)

Number of columns using shape: 19


Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

A: 46.45

In [9]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,N,142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,N,236,42,1,8.0,0.5,0.5,4.00,0.0,0.3,13.30,0.0,0.0
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,N,166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,N,114,68,2,8.0,0.5,0.5,0.00,0.0,0.3,11.80,2.5,0.0
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,N,68,163,1,23.5,0.5,0.5,3.00,0.0,0.3,30.30,2.5,0.0


In [72]:
# Create new duration field in minutes
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

In [73]:
df.duration.describe(percentiles=[0.95,0.98,0.99])

count    2.463931e+06
mean     1.421220e+01
std      4.644531e+01
min     -3.442400e+03
50%      1.018333e+01
95%      3.193333e+01
98%      4.215000e+01
99%      5.085000e+01
max      8.513183e+03
Name: duration, dtype: float64

Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

A: 98%

In [74]:
# Select the data which belongs to 98%/99% percentile
df = df[(df.duration >= 1) & (df.duration <= 60)]

df.duration.describe(percentiles=[0.95,0.98,0.99])


count    2.421440e+06
mean     1.267128e+01
std      8.999282e+00
min      1.000000e+00
50%      1.023333e+01
95%      3.105000e+01
98%      3.971667e+01
99%      4.568333e+01
max      6.000000e+01
Name: duration, dtype: float64

In [15]:
2.421440e+06 / 2.463931e+06

0.9827547930522406

Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

    Turn the dataframe into a list of dictionaries
    Fit a dictionary vectorizer
    Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

A: 515

In [75]:
dv = DictVectorizer()
categorical = ['PULocationID','DOLocationID']
df[categorical] = df[categorical].astype(str)

# Create train dictonary and vectorize it
train_dicts = df[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

In [76]:
# Print the dimensionality of the feature matrix
X_train.shape[1]

515

Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

    Train a plain linear regression model with default parameters
    Calculate the RMSE of the model on the training data

What's the RMSE on train?

A: 6.99

In [77]:
# Define target
target = 'duration'
y_train = df[target].values

In [78]:
#Train the model and predict
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

In [79]:
# Look into RMSE
mean_squared_error(y_train, y_pred, squared=False)

6.986190814952337

Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2022).

What's the RMSE on validation?

A: 7.79

In [80]:
df_val = pd.read_parquet('./data/yellow_tripdata_2022-02.parquet')

# Create new duration field in minutes
df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)
    
# Select the data which belongs to 98%/99% percentile
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

#Error with the fit transform for the validation df
The issue here may be related to the use of the fit_transform() function on your validation set.

The fit_transform() function first fits the vectorizer to the data and then transforms the data.
When you fit the vectorizer to your validation set, it learns a new vocabulary,
which may be different from the one it would learn from the training set.
In particular, if the validation set doesn't contain all the categories present in the training set for the given features,
this could lead to a difference in the number of columns when the data is one-hot encoded.

The common practice is to fit() the vectorizer on the training data and then use transform()
on the validation and test data.
This way, the same vocabulary (i.e., the same set of categories) is used across all datasets.

CODE:
#Fit vectorizer on training data (replace df_train with your actual training data)
train_dicts = df[categorical].to_dict(orient='records')
dv.fit(train_dicts)

#Transform validation data
X_val = dv.transform(val_dicts)

#Print the dimensionality of the feature matrix
print(X_val.shape[1])

In [81]:
# Define Categorical
categorical = ['PULocationID','DOLocationID']
df_val[categorical] = df_val[categorical].astype(str)

# Create validation dictonary and vectorize it
val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

# Define target
target = 'duration'
y_val = df_val[target].values

In [82]:
# Print the dimensionality of the feature matrix
X_val.shape[1]

515

In [83]:
# Fit the model
y_val_pred = lr.predict(X_val)

In [84]:
# Look into RMSE
mean_squared_error(y_val, y_val_pred, squared=False)

7.786407163179794